## Speech recognition

This week’s assignment is a simplified form of speech recognition. The languages are artificial and have been generated using a combination of Markov models and hidden Markov models (HMMs). In real life things are much more messy! This artificial data makes for a better tutorial with cleaner results. There are two forms of the datasets available. In the first form, there are several audio files, which can be parsed into discrete phonemes. In the second form, the parsing has already been done for you, and you are presented with long sequences of symbols. It is worth listening to the audio yourself, and seeing if you can determine any differences between the “languages” or “speakers” by ear! If you want to process the audio dataset, then it is advised to use: scipy.io.wavfile to read an audio file. All audio will be single channel (mono) and noiselessly generated from a small set of component sounds.

In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
## Data Preprocessing ##

text = ''
with open('speaker.txt') as f:
    text = f.read()

o, letters = pd.factorize(list(text)) #returns observed states and unique letters 
print('unique letters ==',letters)

#transition matrix
#these are row stochastic (the rows must sum to 1)
A = np.array([
    [.9,.05,.05],
    [.05,.9,.05],
    [.05,.05,.9]
]) 

#emission matrix
B = [] #uniformly distrtibuted? 

#initial conditions of len 3 because there are three speakers
#set initial conditions to uniform


unique letters == ['e' 'o' 'g' 'A' 'p' 't' 'k']


### 1 - Language detection
There are three languages: A, B, and C. Each language uses the same set of symbols: “A, o, e, t, p, g, and k. However, each language uses the symbols differently. In each of these languages we can model everything as P(next symbol | current symbol).

There is training data available for each language. This consists of several files each generated by sampling from a Markov model. Using python, build a Markov model for each of the languages.
Now use the Markov model and Bayes’ rule to classify the test cases. Write down how you used Bayes’ rule to get your classifier. Give the full posterior distribution for each test case.
Audio dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/nglEdY/audio.zip

Symbol dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/ryDvKV/symbol.zip

In [55]:
#get file names from paths
path = '/Users/anosharahim/cs156-pcw-anosharahim/23/'
trainA = os.listdir(path + 'trainA')
trainB = os.listdir(path + 'trainB')
trainC = os.listdir(path + 'trainC')
ts = os.listdir(path + 'test')

#unique alphabet in language
alphabet = ['A', 'o', 'e', 't', 'p', 'g', 'k']
#create alphabet encoding
alphabet_dict = {'o': 0, 'p': 1, 't': 2, 'e': 3, 'k': 4, 'g': 5, 'A': 6} #

def getfiles(folder,folder_name, pth): 
    items = []
    c=0
    for item in folder: 
        myfile = open(pth+folder_name+'/'+item)
        c+=1
        
        for line in myfile:
            items.append(line)
    print(c)   
    return items

print('training data set sizes')
A_tr = getfiles(trainA, 'trainA', path)
trainB.pop(18) #remove .DStore file
B_tr = getfiles(trainB, 'trainB', path)
C_tr = getfiles(trainC, 'trainC', path)
print('test data size')
test = getfiles(ts, 'test', path)
print('alphabet ==',alphabet)


training data set sizes
30
30
30
test data size
10
alphabet == ['A', 'o', 'e', 't', 'p', 'g', 'k']


In [48]:
def T_matrix(sequences, dict_):
    '''Returns transition matrix of a list of sequences. 
    
    Input:
    
    sequences: list of str
    A list of sequences as strings.
    dict_: dict()
    dictionary of letters as keys and their encoding as values.
    
    Output 
    
    M: list of lists
    where M[i][j] is the probability of transitioning from i to j
    
    '''
    
    #initialize transition matrix
    M = [[0]*7 for _ in range(7)] 
    
    for seq in sequences: #for each sequence
        seq = list(seq) #create list from string
        T = [dict_[letter] for letter in seq] 
        
        for (i,j) in zip(T,T[1:]): #count them 
            M[i][j] += 1 #append count when state transitions from i to j
            
    #one counted, convert to probabilities
    for row in M:
        n = sum(row) #row should sum to 1
        if n > 0:
            #normalize rows
            row[:] = [f/sum(row) for f in row]
            
    return M #transition matrix


def log_probability(seq, t_matrix, letters = alphabet):
    '''Returns log probability that a sequence is from a given language.
    '''
    log_prob = 0 #initialize log probability 
    
    for i in range(len(seq)-1): #for each sequence
        
        #get current and next letter
        curr_ = letters.index(seq[i])
        next_ = letters.index(seq[i+1])
        #find probability of going from curr to next
        #in the transition matrix
        prob = t_matrix[curr_, next_]
        
        #check if probability is greater than 0
        if prob > 0:
            log_prob += np.log(prob)
        
    return log_prob 


def language_detector(seq, t_matrices, letters = alphabet):
    lang_p = []
    for name, t_matrix in t_matrices:
        lang_p.append(log_probability(seq, t_matrix, letters))
    return lang_p    

In [56]:
transition_matrix_A = T_matrix(A_tr, alphabet_dict)
transition_matrix_B = T_matrix(B_tr, alphabet_dict)
transition_matrix_C = T_matrix(C_tr, alphabet_dict)

transition_matrices = [
    ('Language A', transition_matrix_A),
    ('Language B', transition_matrix_B),
    ('Language C', transition_matrix_C)
]

results = pd.DataFrame()


for i in range(len(test)):
    log_probs = language_detector(test[i], transition_matrices, alphabet)
    results = results.append(
        {
            "string": f"test_string_{i}",
            "language_A": log_probs[0],
            "language_B": log_probs[1],
            "language_C": log_probs[2]
        },
        ignore_index=True
    )
    most_probable = np.argmax(np.asarray(log_probs))
    pred_language = transition_matrices[most_probable][0]
    print(f"For the string {i}, prediction is: {pred_language}")

TypeError: list indices must be integers or slices, not tuple

### 2 - Speaker identification
There are three people in a room. Each says about 10 phonemes, before being randomly interrupted by someone else. When they speak they all sound the same, however each person tends to use different phonemes in their speech. Specifically we can model the following transition probabilities that someone will interrupt the current speaker: P(speaker i at time t+1 | speaker j at time t). We can also model the probability over phonemes given a particular speaker: P(phoneme | speaker i). The phonemes are identical to the ones introduced in problem 1 (but the transition matrices are obviously different, since they take a different form altogether).

Write down the update equations that you will need to train a hidden Markov model. Using the information given above, write down a sensible initialization for the transition matrix.
Write your own python code to train a hidden Markov model on the data. You may look at code online, but will need to reference any code that helps you with your implementation.
From matplotlb use a stackplot (https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.stackplot.html) to show the probability of a particular person speaking.
Audio dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/VW8Rjr/speaker.wav.zip Symbol dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/n705lY/speaker

In [5]:
## Calculate Parameters Alpha and Beta ##

def get_alphas(A,B,os,pi):
    '''
    in:
    A - transition matrix
    B - emission matrix
    os - observations
    pi - initial condition
    
    out:
    alphas
    '''
    #use formula: \alpha_t=[\alpha_{t-1}*A]*(b)^T(o_t)
    
    #initiate array to store alphas 
    #of size no. of states x observations 
    alphas = np.zeros((len(pi),len(os))) 
    
    #initialize the first row of alpha using initial conditions
    #so that it has the previous time step value in the loop 
    A[:,0] = pi * B[:,0]
        
    for t in range(1,len(os)): #for each observations, comput alpha
        alphas[:,t] = (A.T @ alphas[:,t-1]) * B[:,os[t]] 
    
    return alphas

def get_betas(A,B,os,pi):
    #use formula: \beta_t-1=[A*b(o_{t+1}]*\beta_{t+1}
    
    betas = np.zeros((len(pi),len(os))) 
    
    for t in range(1,len(os)): #count backwards!!
        #calculate previous beta from the next time step
        betas[:,t] = (A @ B[:,os[t+1]]) * betas[:,t+1]
    
    return betas

In [ ]:
# Baum Welch Algorithm ##

def E_step(A,B,os,pi):
    #use alphas and betas in E step 
    alpha = get_alphas(A,B,os,pi)
    beta = get_betas(A,B,os,pi)
    
    #- We can find P(o|gamma) by marginalizing all possible hmm chains of hidden variables x as such:   
    #$$p(o|\gamma)=\Sigma_{x}p(o|x,\gamma)p(x|\gamma)$$
    marginal = #sum of all somethings
    
    #use formula: \gamma_t(i)=\frac{\alpha_t(i)*\beta_t(i)}{p(o|\gamma)}
    gamma = (alpha*beta)/marginal
    
    #get new pis now
    #pis = do something
    return gamma, new_pis

def M_step(gamma,new_pis):
    
    #get new transition and emission matrix
    #get new_pi using gamma
    
    return

#run the E-M algorithm